# RAG using LangChain

This notebook demonstrates a Retrieval-Augmented Generation (RAG) pipeline using LangChain.

## Objectives
- Load and embed documents
- Store in a FAISS vector store using LangChain
- Retrieve context from the store
- Use a language model to answer questions based on context

### Without LangChain
- You do everything yourself:
    - Manually split documents
    - Manually embed them
    - Manually build and search the FAISS index
    - Manually construct prompts for OpenAI
    - Manually handle context formatting and errors

In [3]:
# Install LangChain and dependencies
%pip install langchain langchain-community faiss-cpu openai sentence-transformers -q

Note: you may need to restart the kernel to use updated packages.


## Step 1: Load Documents and Create Embeddings

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI

# Sample data
text = """The mitochondria is the powerhouse of the cell.
Photosynthesis occurs in the chloroplasts of plant cells.
DNA is stored in the nucleus.
Proteins are synthesized by ribosomes.
ATP provides energy for cellular processes."""

# Split into documents
text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=0)
documents = text_splitter.create_documents([text])

# Embedding model
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create FAISS vector store
db = FAISS.from_documents(documents, embedding)
print("Stored documents:", len(documents))

/home/ola/webage/ai-bah/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: CUDA error: operation not supported
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


## Step 2: Create Retriever and Ask Questions

In [ ]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 2})

query = "What produces energy in cells?"
docs = retriever.get_relevant_documents(query)

print("Top matching chunks:")
for d in docs:
    print("-", d.page_content)

## Step 3: RAG Chain with OpenAI LLM

In [ ]:
from langchain.chains import RetrievalQA

llm = OpenAI(temperature=0.3)

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

result = rag_chain({"query": query})
print("Answer:", result['result'])

## Summary
- LangChain simplified our RAG workflow.
- We created a retriever from vector data.
- Used LangChain’s `RetrievalQA` to fetch relevant info and answer questions.